In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import when

In [2]:
# Initialisation de la session Spark avec Hive
spark = SparkSession.builder \
    .appName("Analyse Immatriculations") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# Utilisation de la base Hive
spark.sql("USE concessionnaire")

# Chargement des données d'immatriculations
immatriculation_df = spark.sql("SELECT * FROM immatriculations_data")

# Afficher un aperçu des données
immatriculation_df.show(10)


+---------------+----------+--------------+---------+-----------+--------+--------+-------+--------+-------+
|immatriculation|    marque|           nom|puissance|   longueur|nbplaces|nbportes|couleur|occasion|   prix|
+---------------+----------+--------------+---------+-----------+--------+--------+-------+--------+-------+
|immatriculation|    marque|           nom|     NULL|   longueur|    NULL|    NULL|couleur|    NULL|   NULL|
|     3176 TS 67|   Renault|   Laguna 2.0T|      170|     longue|       5|       5|  blanc|   false|27300.0|
|     3721 QS 49|     Volvo|        S80 T6|      272|tr�s longue|       5|       5|   noir|   false|50500.0|
|     9099 UV 26|Volkswagen|  Golf 2.0 FSI|      150|    moyenne|       5|       5|   gris|    true|16029.0|
|     3563 LA 55|   Peugeot|      1007 1.4|       75|     courte|       5|       5|  blanc|    true| 9625.0|
|     6963 AX 34|      Audi|        A2 1.4|       75|     courte|       5|       5|   gris|   false|18310.0|
|     5592 HQ 89|  

In [3]:

# Transformer 'Longueur' en données numériques
features_df = immatriculation_df.select(
    col("Puissance").cast("double"),
    col("Prix").cast("double"),
    col("NbPlaces").cast("double"),
    when(col("Longueur") == "courte", 1)
      .when(col("Longueur") == "moyenne", 2)
      .when(col("Longueur") == "longue", 3)
      .when(col("Longueur") == "très longue", 4).alias("Longueur_numeric")
)

# Afficher un aperçu des données transformées
features_df.show(10)


+---------+-------+--------+----------------+
|Puissance|   Prix|NbPlaces|Longueur_numeric|
+---------+-------+--------+----------------+
|     NULL|   NULL|    NULL|            NULL|
|    170.0|27300.0|     5.0|               3|
|    272.0|50500.0|     5.0|            NULL|
|    150.0|16029.0|     5.0|               2|
|     75.0| 9625.0|     5.0|               1|
|     75.0|18310.0|     5.0|               1|
|    193.0|31790.0|     5.0|            NULL|
|    135.0|22350.0|     5.0|               2|
|    136.0|18130.0|     5.0|               2|
|    150.0|25060.0|     5.0|               2|
+---------+-------+--------+----------------+
only showing top 10 rows



In [4]:

# Assembleur pour créer une colonne 'features' pour le clustering
assembler = VectorAssembler(inputCols=["Puissance", "Prix", "NbPlaces", "Longueur_numeric"], outputCol="features")
assembled_df = assembler.transform(features_df)

# Normalisation des données
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(assembled_df)
scaled_df = scaler_model.transform(assembled_df)

# Afficher un aperçu des données normalisées
scaled_df.select("features", "scaled_features").show(10, truncate=False)


Py4JJavaError: An error occurred while calling o73.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (dc9185891147 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3111/0x000000e0020669e8`: (struct<Puissance:double,Prix:double,NbPlaces:double,Longueur_numeric_double_VectorAssembler_c8e31cf58ac3:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIte

In [ ]:

# Appliquer k-means avec 4 clusters
kmeans = KMeans(k=4, seed=42, featuresCol="scaled_features", predictionCol="Cluster")
kmeans_model = kmeans.fit(scaled_df)
clustered_df = kmeans_model.transform(scaled_df)

# Afficher un aperçu des données avec les clusters
clustered_df.select("Puissance", "Prix", "NbPlaces", "Longueur_numeric", "Cluster").show(10)


In [ ]:

# Mapper les clusters à des catégories
clustered_df = clustered_df.withColumn(
    "Catégorie",
    when(col("Cluster") == 0, "Citadine")
    .when(col("Cluster") == 1, "Familiale")
    .when(col("Cluster") == 2, "Sportive")
    .when(col("Cluster") == 3, "Luxe")
)

# Afficher les données avec les catégories
clustered_df.select("Puissance", "Prix", "NbPlaces", "Longueur_numeric", "Cluster", "Catégorie").show(10)


In [ ]:
# Sauvegarder dans une table Hive


print("Catégorisation sauvegardée dans la table 'vehicules_categorises'.")
